# Demonstrating the reusability of [importnb](https://github.com/deathbeds/importnb).

* `importnb` imports notebooks as modules
* `importnb` allows notebooks to be discovered as tests.

    >>> with Notebook(): import test_notebook as nb
    >>> assert 'test_function' in dir(nb)
    >>> assert nb.notebook_to_df()

# This notebooks loads in a directory of notebooks as dataframe.

In [9]:
    from toolz.curried import *
    from toolz.curried.operator import *
    from pathlib import Path
    from pandas import Series, DataFrame, concat as Concat

    ignores = 'checkpoint',

    from nbformat.v4 import reads

    def notebook_directory_to_df(location=''):
        """Convert a directory of notebooks in dataframes.

        returns a dataframe for the cells and for the metadata.
        """
        cells = metadata = None

        suffix = '.ipynb'
        recursive = False
        for path in getattr(Path(location), 'glob')(f"*{recursive and 'r' or ''}{suffix}"):
            for ignore in ignores:
                if ignore in str(path): continue

            nb = pipe(path.read_text(), reads)

            s = DataFrame({k: v for k, v in nb.items() if k !='cells'})
            s = s.metadata.apply(Series).stack().append(Concat({'nbformat': s.iloc[0][['nbformat', 'nbformat_minor']]}))

            if cells is None:
                cells = Concat({path: DataFrame(nb['cells'])})
                metadata = Concat({path: s})
            else:
                cells = cells.append(Concat({path: DataFrame(nb['cells'])}))
                metadata = metadata.append(Concat({path: s}))
        return cells, metadata.unstack([-2, -1])

### [Computational Color](http://printingcode.runemadsen.com/lecture-color/)

In [10]:
    from importnb import Notebook

In [11]:
    __all__ = 'test_function',

In [12]:
    from poser import *

# ipywidget

In [13]:
    @λ.ipywidgets.interact()
    def widget_demo(a=10):
        ...

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), Output()), _dom_classes=('widget-…

## pytest will discover `test_function` because it starts with `test_*`

In [6]:
    def test_function():
        assert True

## doctest can test the current notebook state.

In [7]:
    if __name__ == '__main__':
        import doctest
        doctest.testmod()

## doctest can test the future state of the module.

In [8]:
    if __name__ == '__main__':
        with Notebook():
            import test_notebook as nb
            doctest.testmod(nb)

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), Output()), _dom_classes=('widget-…

**********************************************************************
File "/Users/tonyfast/_what_is_a_hypothesis/test_notebook.ipynb", line 9, in test_notebook
Failed example:
    assert nb.notebook_to_df()
Exception raised:
    Traceback (most recent call last):
      File "/Users/tonyfast/anaconda/envs/p6/lib/python3.6/doctest.py", line 1330, in __run
        compileflags, 1), test.globs)
      File "<doctest test_notebook[2]>", line 1, in <module>
        assert nb.notebook_to_df()
    AttributeError: module 'test_notebook' has no attribute 'notebook_to_df'
**********************************************************************
1 items had failures:
   1 of   3 in test_notebook
***Test Failed*** 1 failures.
